Clay Tryon, HW week 5

Import necessary modules

In [77]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report



Import and save Dataset 

In [5]:
raw_heart_data=pd.read_csv('heart.csv')

In [6]:
print(raw_heart_data.head())

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   0     3       0  
2   0     3       0  
3   1     3       0  
4   3     2       0  


In [7]:
print(raw_heart_data)

      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0      52    1   0       125   212    0        1      168      0      1.0   
1      53    1   0       140   203    1        0      155      1      3.1   
2      70    1   0       145   174    0        1      125      1      2.6   
3      61    1   0       148   203    0        1      161      0      0.0   
4      62    0   0       138   294    1        1      106      0      1.9   
...   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
1020   59    1   1       140   221    0        1      164      1      0.0   
1021   60    1   0       125   258    0        0      141      1      2.8   
1022   47    1   0       110   275    0        0      118      1      1.0   
1023   50    0   0       110   254    0        0      159      0      0.0   
1024   54    1   0       120   188    0        1      113      0      1.4   

      slope  ca  thal  target  
0         2   2     3       0  
1         0

Clean The data. Check for Null values

In [12]:
raw_heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

No nulls found
Check for Duplicate rows

In [14]:
print(raw_heart_data.duplicated().sum())

723


In [18]:
dupes = raw_heart_data[raw_heart_data.duplicated(keep=False)]
print(dupes.sort_values(by=list(raw_heart_data.columns)).head(10))
#I did this to verify that the duplicated secions actually ARE duplicated before dropping them


     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
60    29    1   1       130   204    0        0      202      0      0.0   
64    29    1   1       130   204    0        0      202      0      0.0   
118   29    1   1       130   204    0        0      202      0      0.0   
668   29    1   1       130   204    0        0      202      0      0.0   
12    34    0   1       118   210    0        1      192      0      0.7   
15    34    0   1       118   210    0        1      192      0      0.7   
779   34    0   1       118   210    0        1      192      0      0.7   
143   34    1   3       118   182    0        0      174      0      0.0   
201   34    1   3       118   182    0        0      174      0      0.0   
572   34    1   3       118   182    0        0      174      0      0.0   

     slope  ca  thal  target  
60       2   0     2       1  
64       2   0     2       1  
118      2   0     2       1  
668      2   0     2       1  
12      

In [40]:
trimmed_heart_data=raw_heart_data.drop_duplicates(keep='first').reset_index()

In [41]:
print(trimmed_heart_data)

     index  age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  \
0        0   52    1   0       125   212    0        1      168      0   
1        1   53    1   0       140   203    1        0      155      1   
2        2   70    1   0       145   174    0        1      125      1   
3        3   61    1   0       148   203    0        1      161      0   
4        4   62    0   0       138   294    1        1      106      0   
..     ...  ...  ...  ..       ...   ...  ...      ...      ...    ...   
297    723   68    0   2       120   211    0        0      115      0   
298    733   44    0   2       108   141    0        1      175      0   
299    739   52    1   0       128   255    0        1      161      1   
300    843   59    1   3       160   273    0        0      125      0   
301    878   54    1   0       120   188    0        1      113      0   

     oldpeak  slope  ca  thal  target  
0        1.0      2   2     3       0  
1        3.1      0   0     3  

We now need to check for outliers
any values that are more than 3 std's from the mean are considered an outlier based on the empirical rule in stats


In [42]:
trimmed_heart_data.describe()

,index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,302.000000,302.00000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000
mean,236.304636,54.42053,0.682119,0.963576,131.602649,246.500000,0.149007,0.526490,149.569536,0.327815,1.043046,1.397351,0.718543,2.314570,0.543046
std,188.588990,9.04797,0.466426,1.032044,17.563394,51.753489,0.356686,0.526027,22.903527,0.470196,1.161452,0.616274,1.006748,0.613026,0.498970
min,0.000000,29.00000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,85.500000,48.00000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.250000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,189.500000,55.50000,1.000000,1.000000,130.000000,240.500000,0.000000,1.000000,152.500000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,342.750000,61.00000,1.000000,2.000000,140.000000,274.750000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,878.000000,77.00000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [43]:
outlier_indices = set()
for col in trimmed_heart_data:
    mean=trimmed_heart_data[col].mean()
    std=trimmed_heart_data[col].std()
    lower_bound= mean-(3 * std)
    upper_bound= mean+ (3*std)

    outliers = trimmed_heart_data[(trimmed_heart_data[col] <lower_bound) | (trimmed_heart_data[col] >upper_bound)]
    outlier_indices.update(outliers.index)
    print(f"{col} has {len(outliers)} outliers")
    
print(f'Number of rows containing outliers {len(outlier_indices)}')
print(outlier_indices)

index has 2 outliers
age has 0 outliers
sex has 0 outliers
cp has 0 outliers
trestbps has 2 outliers
chol has 4 outliers
fbs has 0 outliers
restecg has 0 outliers
thalach has 1 outliers
exang has 0 outliers
oldpeak has 2 outliers
slope has 0 outliers
ca has 4 outliers
thal has 2 outliers
target has 0 outliers
Number of rows containing outliers 17
{128, 227, 260, 104, 300, 301, 142, 108, 144, 49, 188, 51, 269, 14, 212, 124, 63}


There are outliers present in the dataset, so we could proceed in one of two ways.

Option 1 is split the data now to test/train, and remove outliers from the training set. This will prevent outliers from skewing our model, while leaving possible edge cases in the test prediction

option 2 is remove all outliers, than split and train. I think this would be too clinical, and would lead to a less accurate model because edge cases DO exist in real life. I am going to go with Option 1 for this assignment

In [67]:
#Drop the target column from the frame before running through model

X = trimmed_heart_data.drop('target', axis=1)
y = trimmed_heart_data['target'] 

In [68]:
#Split the data Set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [69]:
print(train_data)

     index  age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  \
132    163   48    1   0       124   274    0        0      166      0   
202    291   58    1   0       128   259    0        0      130      1   
196    280   45    0   1       130   234    0        0      175      0   
75      85   44    1   1       120   220    0        1      170      0   
176    239   62    0   0       150   244    0        1      154      1   
..     ...  ...  ...  ..       ...   ...  ...      ...      ...    ...   
188    267   67    1   0       120   237    0        1       71      0   
71      77   63    1   0       140   187    0        0      144      1   
106    125   60    0   3       150   240    0        1      171      0   
270    522   67    0   2       152   277    0        1      172      0   
102    119   42    1   1       120   295    0        1      162      0   

     oldpeak  slope  ca  thal  target  
132      0.5      1   0     3       0  
202      3.0      1   2     3  

In [70]:
#Remove outliers from training data

safe_outliers = outlier_indices.intersection(train_data.index)
train_data_clean = train_data.drop(index=safe_outliers)

In [71]:
train_data_clean.reset_index()

,level_0,index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,132,163,48,1,0,124,274,0,0,166,0,0.5,1,0,3,0
1,202,291,58,1,0,128,259,0,0,130,1,3.0,1,2,3,0
2,196,280,45,0,1,130,234,0,0,175,0,0.6,1,0,2,1
3,75,85,44,1,1,120,220,0,1,170,0,0.0,2,0,2,1
4,176,239,62,0,0,150,244,0,1,154,1,1.4,1,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,20,21,67,0,0,106,223,0,1,142,0,0.3,2,2,2,1
222,71,77,63,1,0,140,187,0,0,144,1,4.0,2,2,3,0
223,106,125,60,0,3,150,240,0,1,171,0,0.9,2,0,2,1
224,270,522,67,0,2,152,277,0,1,172,0,0.0,2,1,2,1


Alright. Looking at the target column, its clear this is our predictor of heart disease. Based on the HW, a 1 means disease is present. I need to scale the data before running the model, otherwise large numbers like cholesterol will override numbers like age


In [72]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [74]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [75]:
# Get coefficients
coefficients = model.coef_[0]
intercept = model.intercept_[0]

# Create a readable table
feature_weights = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': coefficients
}).sort_values(by='Coefficient', key=abs, ascending=False)

print("Intercept:", intercept)
display(feature_weights)


Intercept: 0.27972655978235056


,Feature,Coefficient
3,cp,1.004576
2,sex,-0.840883
10,oldpeak,-0.790273
12,ca,-0.586122
4,trestbps,-0.533444
8,thalach,0.480072
7,restecg,0.467827
13,thal,-0.407366
11,slope,0.351559
9,exang,-0.257821


Evaluate the Dataset


In [76]:
y_pred=model.predict(X_val_scaled)

In [78]:
print('Confusion Matrix:')
print(confusion_matrix(y_val, y_pred))

Confusion Matrix:
[[20 12]
 [ 4 25]]


I'm pretty unhappy with this. This works out to about 74% correct, since it is 20 true negatives, 12 false positives, 4 false negatives, and 25 true positives. Yeah it got 45 peoples data correct, but the remaining 16 is a huge margin of error. I think what I would likely do to improve the model, is try training the model again with some of the less important factors from the training data. For example, I didn't even realize the Index column was having weight on the data until just now. Let me try running stuff again for an improved model


In [82]:
X = X.drop(['index', 'fbs', 'age'], axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_val_scaled)

print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))




[[22 10]
 [ 4 25]]
              precision    recall  f1-score   support

           0       0.85      0.69      0.76        32
           1       0.71      0.86      0.78        29

    accuracy                           0.77        61
   macro avg       0.78      0.77      0.77        61
weighted avg       0.78      0.77      0.77        61



This newer model has a slightly higher rate of accuracy, at 77%. I like that it has gotten better with true negatives, and lowered the false positives. 